# importing useful libraries

In [ ]:
!pip install zoofs

In [ ]:
# Math & data
import numpy as np
import pandas as pd

# Plotting
import matplotlib.pyplot as plt
%matplotlib inline
# Image processing
import cv2
import skimage.feature as sk
# Statistics
from scipy import interp
import skimage
from skimage import img_as_ubyte
# Machine learning
from sklearn import preprocessing
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold, GridSearchCV
# Misc
import os
import collections
from itertools import cycle
import glob
from skimage.filters import threshold_otsu
from skimage import io,exposure

# mounting google drive for data access and making required directories

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
if not os.path.exists('/content/total_images'):
  os.makedirs('/content/total_images')

In [ ]:
from zipfile import ZipFile          
file_name = '/content/drive/MyDrive/Infection Segmentation Data.zip'
with ZipFile(file_name,'r') as zipi:
  zipi.extractall('/content/total_images')
  print('Done')

In [ ]:
!rm -rf '/content/total_images/Infection Segmentation Data/Test'

In [ ]:
!rm -rf '/content/total_images/Infection Segmentation Data/Val'

In [ ]:
!rm -rf '/content/total_images/Infection Segmentation Data/Train/Non-COVID'

In [ ]:
covid_images = []
for directory_path in glob.glob('/content/total_images/Infection Segmentation Data/Train/COVID-19/images/*'):
    img = cv2.imread(directory_path, cv2.IMREAD_GRAYSCALE)
    covid_images.append(img)

In [ ]:
len(covid_images) 

In [ ]:
plt.imshow(covid_images[1],cmap = 'gray')

In [ ]:
covid_masks = []
for directory_path in glob.glob('/content/total_images/Infection Segmentation Data/Train/COVID-19/lung masks/*'):
    img = cv2.imread(directory_path, cv2.IMREAD_GRAYSCALE)
    covid_masks.append(img)

In [ ]:
len(covid_masks)

In [ ]:
normal_images = []
for filename in glob.glob('/content/total_images/Infection Segmentation Data/Train/Normal/images/*.png'):
  img = io.imread(filename)
  normal_images.append(img)

In [ ]:
len(normal_images)

In [ ]:
normal_mask = []
for filename in glob.glob('/content/total_images/Infection Segmentation Data/Train/Normal/lung masks/*.png'):
  img = io.imread(filename)
  normal_mask.append(img)

In [ ]:
covid_images_segmented = []
for img1, img2 in zip(covid_images[0:200],covid_masks[0:200]):
  img3 = img1.copy()
  img3[img2==0]=0
  covid_images_segmented.append(img3)

In [ ]:
plt.figure(figsize=(20,20))

for i, image in enumerate(covid_images_segmented[0:100]):
  plt.subplot(10,10,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image, cmap = 'gray')
  
plt.suptitle("Image", fontsize=20)
plt.show()

In [ ]:
normal_images_segmented = []
for img1, img2 in zip(normal_images[0:200],normal_mask[0:200]):
  img3 = img1.copy()
  img3[img2==0]=0
  normal_images_segmented.append(img3)

In [ ]:
plt.figure(figsize=(20,20))

for i, image in enumerate(normal_images_segmented[0:100]):
  plt.subplot(10,10,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image, cmap = 'gray')
  
plt.suptitle("Image", fontsize=20)
plt.show()

In [ ]:
DR = '/content/all_images'
if not os.path.exists(DR):
    os.mkdir(DR)

In [ ]:
import imageio
import skimage
from skimage import img_as_ubyte
Dr = '/content/all_images/covid'
if not os.path.exists(Dr):
    os.mkdir(Dr)
for index, value in enumerate(covid_images_segmented):
  value = img_as_ubyte(value)
  imageio.imwrite(f'/content/all_images/covid/image_{index}.png', np.array(value))
print('done')
    #print(index)

In [ ]:
import imageio
import skimage
from skimage import img_as_ubyte
Dr = '/content/all_images/normal'
if not os.path.exists(Dr):
    os.mkdir(Dr)
for index, value in enumerate(normal_images_segmented):
  value = img_as_ubyte(value)
  imageio.imwrite(f'/content/all_images/normal/image_{index}.png', np.array(value))
print('done')
    #print(index)

In [ ]:
images = []
labels = []
#for directory_path in glob.glob("cell_images/test/*"): 
for directory_path in glob.glob('/content/all_images/*/*'):
    label = directory_path.split("/")[-2]
    #for img_path in glob.glob(os.path.join('/content/ML_lung_data/', '*/*.png')):
    img = io.imread(directory_path)
    img = cv2.resize(img, (256, 256))
    img = skimage.exposure.equalize_adapthist(img)
    #img =skimage.filters.farid(img)
    images.append(img_as_ubyte(img))
    labels.append(label)
      
images = np.array(images)
labels = np.array(labels)

In [ ]:
images1 = []
labels1 = [] 
for directory_path in glob.glob('/content/all_images/*/*'):
    label1 = directory_path.split("/")[-2]
    img1 = io.imread(directory_path)
    img1 = cv2.resize(img1, (256, 256))
    images1.append(img_as_ubyte(img1))
    labels1.append(label1)
      
images1 = np.array(images1)
labels1 = np.array(labels1)

In [ ]:
images1[1].shape,len(images1)

In [ ]:
labels[30:40]

In [ ]:
len(images) 

In [ ]:
segmented_img = []
for img in images1:
    img = cv2.resize(img, (256, 256))
    segmented_img.append(img)

In [ ]:
segmented_img[1].dtype

# viewing the images and masks

In [ ]:
plt.figure(figsize=(20,20))

for i, image in enumerate(images[0:100]):
  plt.subplot(10,10,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image, cmap = 'gray')
  
plt.suptitle("Image", fontsize=20)
plt.show()

In [ ]:
from skimage import exposure,img_as_ubyte
images_mask = []
for img in segmented_img:
  image =exposure.equalize_adapthist(img)
  image = skimage.filters.threshold_otsu(image)
  imag = img>image
  images_mask.append(imag)

In [ ]:
plt.figure(figsize=(20,20))

for i, image in enumerate(images_mask[0:100]):
  plt.subplot(10,10,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image, cmap = 'gray')
  
plt.suptitle("Image", fontsize=20)
plt.show()

# encoding the labels

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(labels)
train_labels_encoded = le.transform(labels)
train_labels_encoded

# feature extraction 

In [ ]:
!pip install pyfeats

In [ ]:
import pyfeats
from pyfeats import textural, multiscale,other

# TEXTURE FEATURES

**1.FIRST_ODER_STATISTICAL_FEATURES**

In [ ]:
labels_FOS = ["FOS_Mean","FOS_Variance","FOS_Median","FOS_Mode","FOS_Skewness",
              "FOS_Kurtosis","FOS_Energy","FOS_Entropy","FOS_MinimalGrayLevel",
              "FOS_MaximalGrayLevel","FOS_CoefficientOfVariation",
              "FOS_10Percentile","FOS_25Percentile","FOS_75Percentile",
              "FOS_90Percentile","FOS_HistogramWidth"]
first_order_statistical = []  #fos
for image, mask in zip(images,images_mask):
  features,label = textural.fos(image,mask)
  first_order_statistical.append(features)



In [ ]:
df_first_order_stats = pd.DataFrame(first_order_statistical,columns = ["FOS_Mean","FOS_Variance","FOS_Median","FOS_Mode","FOS_Skewness",
              "FOS_Kurtosis","FOS_Energy","FOS_Entropy","FOS_MinimalGrayLevel",
              "FOS_MaximalGrayLevel","FOS_CoefficientOfVariation",
              "FOS_10Percentile","FOS_25Percentile","FOS_75Percentile",
              "FOS_90Percentile","FOS_HistogramWidth"])
df_first_order_stats

In [ ]:
df_first_order_stats.to_csv('df_first_order_stats')

**2.GLCM_FEATURES**

In [ ]:
glcm_labels = ["GLCM_ASM", "GLCM_Contrast", "GLCM_Correlation",
              "GLCM_SumOfSquaresVariance", "GLCM_InverseDifferenceMoment",
               "GLCM_SumAverage", "GLCM_SumVariance", "GLCM_SumEntropy",
               "GLCM_Entropy", "GLCM_DifferenceVariance",
               "GLCM_DifferenceEntropy", "GLCM_Information1",
               "GLCM_Information2", "GLCM_MaximalCorrelationCoefficient"]

GLCM = []  #fos
for image in images:
  image = img_as_ubyte(image)
  features,label,feat_mean,mean_label = textural.glcm_features(image,ignore_zeros= True)
  GLCM.append(features)

In [ ]:
df_GLCM = pd.DataFrame(GLCM,columns =["GLCM_ASM", "GLCM_Contrast", "GLCM_Correlation",
              "GLCM_SumOfSquaresVariance", "GLCM_InverseDifferenceMoment",
               "GLCM_SumAverage", "GLCM_SumVariance", "GLCM_SumEntropy",
               "GLCM_Entropy", "GLCM_DifferenceVariance",
               "GLCM_DifferenceEntropy", "GLCM_Information1",
               "GLCM_Information2", "GLCM_MaximalCorrelationCoefficient"])
df_GLCM

In [ ]:
df_GLCM.to_csv('df_GLCM')

**3.Law's Texture Energy Measures (LTE/TEM)**

In [ ]:
labels_LTE =['LTE_LL_7', 'LTE_EE_7', 'LTE_SS_7', 'LTE_LE_7', 'LTE_ES_7', 'LTE_LS_7']  ##lte_measures(f, mask, l=7) default l=7
LTE = []  #fos
for image, mask in zip(images,images_mask):
  #image = img_as_ubyte(exposure.equalize_adapthist(image,clip_limit = 0.01))
  features,label = textural.lte_measures(image,mask)
  LTE.append(features)

In [ ]:
df_LTE = pd.DataFrame(LTE, columns = ['LTE_LL_7', 'LTE_EE_7', 'LTE_SS_7', 'LTE_LE_7', 'LTE_ES_7', 'LTE_LS_7'])
df_LTE

In [ ]:
df_LTE.to_csv('df_LTE')

**4.Gray-Level-Run-Length-Matrix**

In [ ]:
labels_GLRLM = ["GLRLM_ShortRunEmphasis",
              "GLRLM_LongRunEmphasis",
              "GLRLM_GrayLevelNo-Uniformity",
              "GLRLM_RunLengthNonUniformity",
              "GLRLM_RunPercentage",
              "GLRLM_LowGrayLevelRunEmphasis",
              "GLRLM_HighGrayLevelRunEmphasis",
              "GLRLM_Short owGrayLevelEmphasis",
              "GLRLM_ShortRunHighGrayLevelEmphasis",
              "GLRLM_LongRunLowGrayLevelEmphasis",
              "GLRLM_LongRunHighGrayLevelEmphasis"]
GLRLM = []  #fos
for image, mask in zip(images,images_mask):
  #image = img_as_ubyte(exposure.equalize_adapthist(image,clip_limit = 0.01))
  features,label = textural.glrlm_features(image,mask)
  GLRLM.append(features)

In [ ]:
df_GLRLM = pd.DataFrame(GLRLM, columns = ["GLRLM_ShortRunEmphasis",
              "GLRLM_LongRunEmphasis",
              "GLRLM_GrayLevelNo-Uniformity",
              "GLRLM_RunLengthNonUniformity",
              "GLRLM_RunPercentage",
              "GLRLM_LowGrayLevelRunEmphasis",
              "GLRLM_HighGrayLevelRunEmphasis",
              "GLRLM_Short owGrayLevelEmphasis",
              "GLRLM_ShortRunHighGrayLevelEmphasis",
              "GLRLM_LongRunLowGrayLevelEmphasis",
              "GLRLM_LongRunHighGrayLevelEmphasis"])
df_GLRLM

In [ ]:
df_GLRLM.to_csv('df_GLRLM')

**5.LBP**

In [ ]:
labels_LBP = ['LBP_R_1_P_8_energy',
 'LBP_R_1_P_8_entropy',
 'LBP_R_2_P_16_energy',
 'LBP_R_2_P_16_entropy',
 'LBP_R_3_P_24_energy',
 'LBP_R_3_P_24_entropy']
LBP = []  
for image, mask in zip(images,images_mask):
  features,label = textural.lbp_features(image,mask,P=[4,8,16,24],R=[1,2,3,4])
  LBP.append(features)

In [ ]:
df_LBP = pd.DataFrame(LBP, columns = ['LBP_R_1_P_4_energy','LBP_R_1_P_4_entropy','LBP_R_2_P_8_energy',
 'LBP_R_2_P_8_entropy',
 'LBP_R_3_P_16_energy',
 'LBP_R_3_P_16_entropy',
 'LBP_R_4_P_24_energy',
 'LBP_R_4_P_24_entropy'])
df_LBP

In [ ]:
df_LBP.to_csv('df_LBP')

# MULTISCALE FEATURES

**2. DWT**

In [ ]:
labels_dwt = ['DWT_bior3.3_level_1_da_mean',
 'DWT_bior3.3_level_1_da_std',
 'DWT_bior3.3_level_1_dd_mean',
 'DWT_bior3.3_level_1_dd_std',
 'DWT_bior3.3_level_1_ad_mean',
 'DWT_bior3.3_level_1_ad_std',
 'DWT_bior3.3_level_2_da_mean',
 'DWT_bior3.3_level_2_da_std',
 'DWT_bior3.3_level_2_dd_mean',
 'DWT_bior3.3_level_2_dd_std',
 'DWT_bior3.3_level_2_ad_mean',
 'DWT_bior3.3_level_2_ad_std',
 'DWT_bior3.3_level_3_da_mean',
 'DWT_bior3.3_level_3_da_std',
 'DWT_bior3.3_level_3_dd_mean',
 'DWT_bior3.3_level_3_dd_std',
 'DWT_bior3.3_level_3_ad_mean',
 'DWT_bior3.3_level_3_ad_std']
DWT = []  #fos
for image, mask in zip(images,images_mask):
  features,label = multiscale.dwt_features(image,mask)
  DWT.append(features)

In [ ]:
df_DWT = pd.DataFrame(DWT,columns = ['DWT_bior3.3_level_1_da_mean',
 'DWT_bior3.3_level_1_da_std',
 'DWT_bior3.3_level_1_dd_mean',
 'DWT_bior3.3_level_1_dd_std',
 'DWT_bior3.3_level_1_ad_mean',
 'DWT_bior3.3_level_1_ad_std',
 'DWT_bior3.3_level_2_da_mean',
 'DWT_bior3.3_level_2_da_std',
 'DWT_bior3.3_level_2_dd_mean',
 'DWT_bior3.3_level_2_dd_std',
 'DWT_bior3.3_level_2_ad_mean',
 'DWT_bior3.3_level_2_ad_std',
 'DWT_bior3.3_level_3_da_mean',
 'DWT_bior3.3_level_3_da_std',
 'DWT_bior3.3_level_3_dd_mean',
 'DWT_bior3.3_level_3_dd_std',
 'DWT_bior3.3_level_3_ad_mean',
 'DWT_bior3.3_level_3_ad_std'])
df_DWT

In [ ]:
df_DWT.to_csv('df_DWT')

# zernikes moment

In [ ]:
labels_zenikes = ['zenikes_0','zenikes_1','zenikes_2','zenikes_3','zenikes_4','zenikes_5','zenikes_6','zenikes_7','zenikes_8','zenikes_9','zenikes_10','zenikes_11','zenikes_12','zenikes_13',
                  'zenikes_14','zenikes_15','zenikes_16','zenikes_17','zenikes_18','zenikes_19','zenikes_20','zenikes_21','zenikes_22','zenikes_23','zenikes_24']
zernikes = []  #fos
for image, mask in zip(images,images_mask):
  features, labels = pyfeats.other.zernikes_moments(image,radius = 9)
  zernikes.append(features)





In [ ]:
labels_zenikes = ['zenikes_0','zenikes_1','zenikes_2','zenikes_3','zenikes_4','zenikes_5','zenikes_6','zenikes_7','zenikes_8','zenikes_9','zenikes_10','zenikes_11','zenikes_12','zenikes_13',
                  'zenikes_14','zenikes_15','zenikes_16','zenikes_17','zenikes_18','zenikes_19','zenikes_20','zenikes_21','zenikes_22','zenikes_23','zenikes_24']
df_zernikes = pd.DataFrame(zernikes,columns = labels_zenikes)
df_zernikes

In [ ]:
df_zernikes.to_csv('df_zernikes')

# checking all csv files containing features

In [ ]:
df_fos = pd.read_csv('/content/df_first_order_stats')
df_LTE = pd.read_csv('/content/df_LTE')
df_LBP = pd.read_csv('/content/df_LBP')
df_GLCM = pd.read_csv('/content/df_GLCM')
df_GLRLM = pd.read_csv('/content/df_GLRLM')
df_DWT = pd.read_csv('/content/df_DWT')
df_zernikes = pd.read_csv('/content/df_zernikes') 


# concatenating the features to make a single dataframe

In [ ]:
df = pd.concat([df_fos.reset_index(drop=True),df_GLCM.reset_index(drop=True),
          df_GLRLM.reset_index(drop=True),df_zernikes.reset_index(drop=True),df_LBP.reset_index(drop = True),df_LTE.reset_index(drop = True),df_DWT.reset_index(drop = True)],axis = 1,join = 'inner')
df.drop(['Unnamed: 0'],axis = 1,inplace=True)
df

In [ ]:
df.loc[:, df.isnull().any()].columns

In [ ]:
df.columns

# scaling the features and splitting into train test part

In [ ]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
scaler = StandardScaler()
data_rescaled = scaler.fit_transform(df)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(reduced,train_labels_encoded, test_size=0.2, random_state=0)

In [ ]:
len(X_train),len(X_test),len(y_train),len(y_test)

# classification without feature optimization to get the best hyperparameters 

# random forest classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier


In [ ]:
rfc = RandomForestClassifier(random_state = 1)

In [ ]:
rfc.fit(X_train, y_train.ravel())
predictions = rfc.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,predictions))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, KFold
kfold =KFold(n_splits=5)

In [ ]:
param_grid = {'n_estimators':[20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,250,300],'max_features':[ 'auto','sqrt', 'log2'],'criterion': ['gini','entropy']}
grid_search = GridSearchCV(rfc,param_grid = param_grid ,n_jobs=-1,scoring = 'accuracy',cv = kfold)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
rfc1 = RandomForestClassifier(criterion='entropy', n_estimators=60, random_state=1)#max_features='auto', n_estimators=1000,warm_start=True)#n_estimators=1000,warm_start=True)
rfc1.fit(X_train,y_train)
predictions1 = rfc1.predict(X_test)
print(classification_report(y_test,predictions1))

In [ ]:
from sklearn.metrics import roc_curve,auc
fpr_RF, tpr_RF, thresholds_RF= roc_curve(y_test, predictions1)
auc_RF = auc(fpr_RF, tpr_RF)


In [ ]:
plt.figure(figsize = (8,8))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_RF, tpr_RF, label='Keras (area = {:.3f})'.format(auc_RF))
#plt.plot(fpr_keras, tpr_keras, label='RF (area = {:.3f})'.format(auc_rf))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

# kNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


In [ ]:
knn = KNeighborsClassifier()

In [ ]:
knn.fit(X_train,y_train.ravel())

In [ ]:
predictions2 = knn.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,predictions2))

In [ ]:
error_rate = []

for i in range(1,60): 
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train.ravel())
    pred_i = knn.predict(X_test)
    error_rate.append(np.mean(pred_i != y_test))

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(range(1,60),error_rate,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')

In [ ]:
knn1 = KNeighborsClassifier(n_neighbors= 2)

In [ ]:
knn1.fit(X_train,y_train)
predictions3 = knn1.predict(X_test)
print(classification_report(y_test,predictions3))

In [ ]:
from sklearn.model_selection import GridSearchCV
kfold =KFold(5)

In [ ]:
grid_params = { 'n_neighbors' : range(1, 50),
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan']}

In [ ]:
gs = GridSearchCV(KNeighborsClassifier(), grid_params, verbose = 1, cv=kfold, n_jobs = -1)

In [ ]:
g_res = gs.fit(X_train, y_train)

In [ ]:
g_res.best_params_

In [ ]:
g_res.best_estimator_

In [ ]:
new_knn = KNeighborsClassifier(metric='manhattan', n_neighbors=2)#, weights='distance')
new_knn.fit(X_train,y_train)
predictions4 = new_knn.predict(X_test)
print(classification_report(y_test,predictions4))

In [ ]:
from sklearn.metrics import roc_curve,auc
fpr_knn, tpr_knn, thresholds_knn = roc_curve(y_test, predictions4)
auc_knn = auc(fpr_knn, tpr_knn)

In [ ]:
plt.figure(figsize = (8,8))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_knn, tpr_knn, label='Knn (area = {:.3f})'.format(auc_knn))
#plt.plot(fpr_keras, tpr_keras, label='RF (area = {:.3f})'.format(auc_rf))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

# SVM

In [ ]:
from sklearn.svm import SVC
svc = SVC()

In [ ]:
svc.fit(X_train, y_train)

In [ ]:
predictions5 = svc.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,predictions5))

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.1, 1, 10, 100, 1000],
			'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
			'kernel': ['rbf','poly']}
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 1,cv = kfold)
grid.fit(X_train, y_train)


In [ ]:
# print best parameter after tuning
print(grid.best_params_)
 
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

In [ ]:
grid_predictions = grid.predict(X_test)
 
# print classification report
print(classification_report(y_test, grid_predictions))

In [ ]:
from sklearn.metrics import roc_curve,auc
fpr_SVC, tpr_SVC, thresholds_SVC = roc_curve(y_test, grid_predictions)
auc_SVC = auc(fpr_SVC, tpr_SVC)

In [ ]:
plt.figure(figsize = (8,8))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_SVC, tpr_SVC, label=' SVC (area = {:.3f})'.format(auc_SVC))
#plt.plot(fpr_keras, tpr_keras, label='RF (area = {:.3f})'.format(auc_rf))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

# LGBM

In [ ]:
import lightgbm as lgb
lgb_model = lgb.LGBMClassifier(random_state=0)  
y_pred_lgb = lgb_model.fit(X_train, y_train).predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred_lgb))

In [ ]:
params_lgb = {
        'bagging_fraction': (0.5, 0.8),
        'bagging_frequency': (5, 8),
        'feature_fraction': (0.5, 0.8),
        'max_depth': (10,11,12,13),
        'min_data_in_leaf': (90,100,110, 120),
        'num_leaves': (1200, 1550)
}
grid_search_lgb = GridSearchCV(lgb_model,param_grid = params_lgb ,n_jobs=-1,scoring = 'accuracy',cv = kfold)
gs_lgb = grid_search_lgb.fit(X_train, y_train)
gs_lgb.best_params_
gs_lgb.best_estimator_

In [ ]:
new_lgb =  lgb.LGBMClassifier(bagging_fraction=0.5, bagging_frequency=5, feature_fraction=0.8,
               max_depth=10, min_data_in_leaf=90, num_leaves=1200,random_state = 0)#, weights='distance')
new_lgb.fit(X_train,y_train)
grid_predictions_lgb = new_lgb.predict(X_test)
print(classification_report(y_test, grid_predictions_lgb))

# optimizers

# feature optimization 

In [ ]:
data_rescaled_df = pd.DataFrame(data_rescaled)
data_rescaled_df.columns = data_rescaled_df.columns.astype(str)
data_rescaled_df.to_csv('data_rescaled_df.csv')

In [ ]:
X_train1, X_test1, y_train1,y_test1 = train_test_split(data_rescaled_df,train_labels_encoded, test_size=0.2, random_state=0)

In [ ]:
from sklearn.metrics import accuracy_score
def objective_function_topass(model,X_train1, y_train1, X_test1, y_test1):      
    model.fit(X_train1,y_train1)  
    P=accuracy_score(y_test1,model.predict(X_test1))
    return P

# Greywolfoptimizer + RandomForest

In [ ]:
from zoofs import GreyWolfOptimization
# create object of algorithm
grey_algo_object=GreyWolfOptimization(objective_function_topass,n_iteration=20,
                                       population_size=40,minimize=False)

In [ ]:
RF_model = RandomForestClassifier(random_state = 1)
RF_model.fit(X_train1, y_train1.ravel())
predictions_RF = RF_model.predict(X_test1)




In [ ]:
param_grid_RF = {'n_estimators':[20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,250,300],'max_features':['auto','sqrt', 'log2'],'criterion': ['gini','entropy'],'bootstrap':[True, False]}
grid_search_RF = GridSearchCV(RF_model,param_grid = param_grid_RF ,n_jobs=-1,scoring = 'accuracy',cv = kfold)
grid_search_RF.fit(X_train1,y_train1.ravel())
grid_search_RF.best_params_
grid_search_RF.best_estimator_


In [ ]:
RF_model = RandomForestClassifier()                                       
RF_object_RF = grey_algo_object.fit(RandomForestClassifier(bootstrap = grid_search_RF.best_params_['bootstrap'],
                                          max_features = grid_search_RF.best_params_['max_features'],
                                          n_estimators = grid_search_RF.best_params_['n_estimators'],
                                          criterion = grid_search_RF.best_params_['criterion'],random_state = 1),X_train1, y_train1, X_test1, y_test1,verbose=True)

#plot your results


In [ ]:
print(list(grey_algo_object.best_feature_list))

In [ ]:
grey_algo_object.plot_history()

In [ ]:
grey_df1_RF = pd.DataFrame(data_rescaled_df, columns=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '14', '16', '17', '18', '19',
                                                      '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '34', '35', '37', 
                                                      '38', '39', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54',
                                                      '55', '56', '57', '58', '59', '60', '61', '62', '64', '65', '66', '67', '68', '69', '70', '72', 
                                                      '73', '74', '76', '77', '80', '81', '83', '85', '86', '87', '88', '89', '90', '91', '92', '93', 
                                                      '94', '95']
 )
grey_df1_RF

In [ ]:
from sklearn.model_selection import train_test_split
X_train_RF, X_test_RF, y_train_RF,y_test_RF = train_test_split(grey_df1_RF,train_labels_encoded, test_size=0.2, random_state=0)

In [ ]:
grey_rfc = RandomForestClassifier(bootstrap=False, max_features='log2', n_estimators=40,
                       random_state=1)   ###grey-random_forest

In [ ]:
grey_rfc.fit(X_train_RF, y_train_RF.ravel())

In [ ]:
grey_predictions = grey_rfc.predict(X_test_RF)


In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test_RF,grey_predictions))

# GreywolfOptimizer + KNN

In [ ]:
KNN_model = KNeighborsClassifier()
KNN_model.fit(X_train1, y_train1.ravel())
predictions_KNN = KNN_model.predict(X_test1)
print(classification_report(y_test1,predictions_KNN))

In [ ]:
grid_params_KNN = { 'n_neighbors' : range(1, 50),
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan']}
gs = GridSearchCV(KNeighborsClassifier(), grid_params_KNN, verbose = 1, cv=kfold, n_jobs = -1)
grid_search_KNN = gs.fit(X_train1, y_train1)
grid_search_KNN.best_params_
grid_search_KNN.best_estimator_

In [ ]:
KNN_object = grey_algo_object.fit(KNeighborsClassifier(n_neighbors = grid_search_KNN.best_params_['n_neighbors'],
                                          weights = grid_search_KNN.best_params_['weights'],
                                          metric = grid_search_KNN.best_params_['metric']),X_train1, y_train1, X_test1, y_test1,verbose=True)

In [ ]:
print(list(grey_algo_object.best_feature_list))

In [ ]:
grey_algo_object.plot_history()

In [ ]:
grey_df1_KNN = pd.DataFrame(data_rescaled_df, columns=['0', '2', '3', '5', '6', '7', '8', '9', '10', '11', '14', '15', '16', '17', '18', '19', 
                                                       '22', '23', '24', '25', '26', '27', '28', '30', '31', '32', '33', '34', '35', '36', '38',
                                                       '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', 
                                                       '54', '55', '56', '58', '59', '60', '61', '63', '64', '65', '66', '67', '68', '69', '70', 
                                                       '71', '72', '73', '74', '75', '76', '77', '79', '80', '81', '82', '83', '84', '85', '86', 
                                                       '88', '89', '90', '91', '92', '93', '94', '95', '96'])
grey_df1_KNN

In [ ]:
X_train_KNN, X_test_KNN, y_train_KNN,y_test_KNN= train_test_split(grey_df1_KNN,train_labels_encoded, test_size=0.2, random_state=0)

In [ ]:
grey_KNN= KNeighborsClassifier(n_neighbors=6, weights='distance')  
grey_KNN.fit(X_train_KNN, y_train_KNN.ravel())
grey_predictions_KNN = grey_KNN.predict(X_test_KNN)
print(classification_report(y_test_KNN,grey_predictions_KNN))

# GreywolfOptimizer + SVM

In [ ]:
SVC_model = SVC(random_state = 0)
SVC_model.fit(X_train1, y_train1.ravel())
predictions_SVC= SVC_model.predict(X_test1)
print(classification_report(y_test1,predictions_SVC))

In [ ]:
grid_params_SVC = {'C': [0.1, 1, 10, 100, 1000],
			'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
			'kernel': ['rbf','poly']}
grid_search_SVC = GridSearchCV(SVC_model, grid_params_SVC, verbose = 1, cv=kfold, n_jobs = -1)
grid_search_SVC= grid_search_SVC.fit(X_train1, y_train1)

In [ ]:
grid_search_SVC.best_params_
grid_search_SVC.best_estimator_

In [ ]:
SVC_object = grey_algo_object.fit(SVC(C = grid_search_SVC.best_params_['C'],
                                          gamma = grid_search_SVC.best_params_['gamma'],
                                          kernel = grid_search_SVC.best_params_['kernel'],random_state = 0),X_train1, y_train1, X_test1, y_test1,verbose=True)

In [ ]:
print(list(grey_algo_object.best_feature_list))

In [ ]:
grey_algo_object.plot_history()

In [ ]:
grey_df1_SVC= pd.DataFrame(data_rescaled_df,columns=['0', '1', '2', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16',
                                                     '18', '21', '23', '24', '25', '28', '29', '31', '32', '34', '35', '36', '38', '40', '41',
                                                     '42', '43', '44', '45', '46', '47', '49', '50', '51', '52', '53', '54', '55', '56', '57', 
                                                     '58', '59', '60', '61', '62', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73',
                                                     '74', '75', '76', '77', '78', '79', '80', '81', '82', '84', '85', '86', '87', '88', '89',
                                                     '90', '93', '94', '96', '97'])
grey_df1_SVC

In [ ]:
X_train_SVC, X_test_SVC, y_train_SVC,y_test_SVC= train_test_split(grey_df1_SVC,train_labels_encoded, test_size=0.2, random_state=0)

In [ ]:
grey_SVC= SVC(C=100, gamma=0.001, random_state=0)  
grey_SVC.fit(X_train_SVC, y_train_SVC.ravel())
grey_predictions_SVC = grey_SVC.predict(X_test_SVC)

In [ ]:
print(classification_report(y_test_SVC,grey_predictions_SVC))

# GreywolfOptimizer + LGB

In [ ]:
import lightgbm as lgb
LGB_model = lgb.LGBMClassifier(random_state = 0)  
y_pred_LGB = LGB_model.fit(X_train1, y_train1).predict(X_test1)
print(classification_report(y_test1,y_pred_LGB))

In [ ]:
grid_params_LGB ={
        'bagging_fraction': (0.5, 0.8),
        'bagging_frequency': (5, 8),
        'feature_fraction': (0.5, 0.8),
        'max_depth': (10,11,12,13),
        'min_data_in_leaf': (90,100,110, 120),
        'num_leaves': (1200, 1550)}
grid_search_LGB = GridSearchCV(LGB_model, grid_params_LGB, verbose = 1, cv=kfold, n_jobs = -1)
grid_search_LGB= grid_search_LGB.fit(X_train1, y_train1)


In [ ]:
grid_search_LGB.best_params_
grid_search_LGB.best_estimator_

In [ ]:
LGB_object = grey_algo_object.fit(lgb.LGBMClassifier(bagging_fraction = grid_search_LGB.best_params_['bagging_fraction'],
                                          bagging_frequency = grid_search_LGB.best_params_['bagging_frequency'],
                                          feature_fraction = grid_search_LGB.best_params_['feature_fraction'],
                                          max_depth = grid_search_LGB.best_params_['max_depth'],
                                          min_data_in_leaf=grid_search_LGB.best_params_['min_data_in_leaf'],
                                          num_leaves=grid_search_LGB.best_params_['num_leaves'],
                                          random_state = 0),X_train1, y_train1, X_test1, y_test1,verbose=True)

In [ ]:
print(list(grey_algo_object.best_feature_list))

In [ ]:
grey_algo_object.plot_history()

In [ ]:
grey_df1_LGB= pd.DataFrame(data_rescaled_df,columns=['0', '2', '3', '4', '5', '6', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '20', '21', '22', 
                                                     '23', '24', '25', '27', '28', '29', '30', '32', '33', '34', '35', '36', '39', '40', '41', '42', '43', 
                                                     '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60',
                                                     '61', '62', '63', '66', '67', '69', '70', '71', '73', '75', '76', '78', '79', '80', '81', '82', '83',
                                                     '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '96', '97'])
grey_df1_LGB

In [ ]:
X_train_LGB, X_test_LGB, y_train_LGB,y_test_LGB= train_test_split(grey_df1_LGB,train_labels_encoded, test_size=0.2, random_state=0)

In [ ]:
grey_LGB= lgb.LGBMClassifier(bagging_fraction=0.5, bagging_frequency=5, feature_fraction=0.5,
               max_depth=10, min_data_in_leaf=110, num_leaves=1200,
               random_state=0)  
grey_LGB.fit(X_train_LGB, y_train_LGB.ravel())
grey_predictions_LGB = grey_LGB.predict(X_test_LGB)
print(classification_report(y_test_LGB,grey_predictions_LGB))

# PSO  + RandomForest

In [ ]:
from zoofs import ParticleSwarmOptimization
# create object of algorithm
PSO_algo_object=ParticleSwarmOptimization(objective_function_topass,n_iteration=20,
                                       population_size=40,minimize=False)

In [ ]:
RFC_object_PSO = PSO_algo_object.fit(RandomForestClassifier(bootstrap = grid_search_RF.best_params_['bootstrap'],
                                          max_features = grid_search_RF.best_params_['max_features'],
                                          n_estimators = grid_search_RF.best_params_['n_estimators'],
                                          criterion = grid_search_RF.best_params_['criterion'],random_state = 1),X_train1, y_train1, X_test1, y_test1,verbose=True)


In [ ]:
print(list(PSO_algo_object.best_feature_list))

In [ ]:
PSO_algo_object.plot_history()

In [ ]:
PSO_df1_RFC= pd.DataFrame(data_rescaled_df, columns=['1', '6', '7', '8', '9', '10', '11', '12', '13', '15', '16', '17', '18', '21', '22', '23', '26', 
                                                     '32', '35', '36', '38', '45', '46', '48', '50', '54', '57', '58', '60', '62', '65', '66', '67',
                                                     '69', '70', '73', '74', '77', '81', '85', '86', '87', '90', '92', '93', '94', '96', '97']
 
 )
PSO_df1_RFC

In [ ]:
X_train_RFC_PSO, X_test_RFC_PSO, y_train_RFC_PSO,y_test_RFC_PSO= train_test_split(PSO_df1_RFC,train_labels_encoded, test_size=0.2, random_state=0)

In [ ]:
PSO_RFC= RandomForestClassifier(bootstrap=False, max_features='log2', n_estimators=40,
                       random_state=1)  
PSO_RFC.fit(X_train_RFC_PSO, y_train_RFC_PSO.ravel())
PSO_predictions_RFC = PSO_RFC.predict(X_test_RFC_PSO)
print(classification_report(y_test_RFC_PSO,PSO_predictions_RFC))

# PSO  + KNN

In [ ]:
KNN_object_PSO = PSO_algo_object.fit(KNeighborsClassifier(n_neighbors = grid_search_KNN.best_params_['n_neighbors'],
                                          weights = grid_search_KNN.best_params_['weights'],
                                          metric = grid_search_KNN.best_params_['metric']),X_train1, y_train1, X_test1, y_test1,verbose=True)

#plot your results


In [ ]:
print(list(PSO_algo_object.best_feature_list))

In [ ]:
PSO_algo_object.plot_history()

In [ ]:
PSO_df1_KNN= pd.DataFrame(data_rescaled_df, columns=['2', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '15', '20', '21', '26', '27', '29', '32',
                                                     '33', '35', '36', '38', '39', '41', '42', '44', '48', '49', '50', '55', '58', '60', '65', '66', '67',
                                                     '72', '73', '74', '75', '77', '78', '85', '88', '92', '93', '94', '95']
 
 )
PSO_df1_KNN

In [ ]:
X_train_KNN_PSO, X_test_KNN_PSO, y_train_KNN_PSO,y_test_KNN_PSO= train_test_split(PSO_df1_KNN,train_labels_encoded, test_size=0.2, random_state=0)

In [ ]:
PSO_KNN= KNeighborsClassifier(n_neighbors=6, weights='distance') 
PSO_KNN.fit(X_train_KNN_PSO, y_train_KNN_PSO.ravel())
PSO_predictions_KNN = PSO_KNN.predict(X_test_KNN_PSO)
print(classification_report(y_test_KNN_PSO,PSO_predictions_KNN))

# PSO  + SVM

In [ ]:
SVC_object_PSO = PSO_algo_object.fit(SVC(C = grid_search_SVC.best_params_['C'],
                                          gamma = grid_search_SVC.best_params_['gamma'],
                                          kernel = grid_search_SVC.best_params_['kernel']),X_train1, y_train1, X_test1, y_test1,verbose=True)

#plot your results


In [ ]:
print(list(PSO_algo_object.best_feature_list))

In [ ]:
PSO_algo_object.plot_history()

In [ ]:
PSO_df1_SVC= pd.DataFrame(data_rescaled_df, columns=['1', '4', '7', '11', '12', '16', '19', '20', '26', '27', '33', '34', '35', '36', '40', '41', 
                                                     '44', '45', '48', '49', '52', '53', '56', '57', '58', '59', '61', '63', '64', '66', '67', '68', 
                                                     '69', '70', '72', '74', '76', '77', '78', '80', '81', '84', '85', '86', '89', '90', '91', '94',
                                                     '95', '97']
 
 )
PSO_df1_SVC

In [ ]:
from sklearn.model_selection import train_test_split
X_train_SVC_PSO, X_test_SVC_PSO, y_train_SVC_PSO,y_test_SVC_PSO= train_test_split(PSO_df1_SVC,train_labels_encoded, test_size=0.2, random_state=0)

In [ ]:
PSO_SVC= SVC(C=100, gamma=0.001, random_state=0)  
PSO_SVC.fit(X_train_SVC_PSO, y_train_SVC_PSO.ravel())
PSO_predictions_SVC = PSO_SVC.predict(X_test_SVC_PSO)
print(classification_report(y_test_SVC_PSO,PSO_predictions_SVC))

# PSO  + LGB

In [ ]:
LGB_object_PSO = PSO_algo_object.fit(lgb.LGBMClassifier(bagging_fraction = grid_search_LGB.best_params_['bagging_fraction'],
                                          bagging_frequency = grid_search_LGB.best_params_['bagging_frequency'],
                                          feature_fraction = grid_search_LGB.best_params_['feature_fraction'],
                                          max_depth = grid_search_LGB.best_params_['max_depth'],
                                          min_data_in_leaf=grid_search_LGB.best_params_['min_data_in_leaf'],
                                          num_leaves=grid_search_LGB.best_params_['num_leaves'],
                                          random_state = 0),X_train1, y_train1, X_test1, y_test1,verbose=True)

#plot your results


In [ ]:
print(list(PSO_algo_object.best_feature_list))

In [ ]:
PSO_algo_object.plot_history()

In [ ]:
PSO_df1_LGB= pd.DataFrame(data_rescaled_df, columns=['1', '9', '10', '11', '12', '13', '14', '16', '17', '20', '21', '22', '23', '24', '25', '26', '29',
                                                     '30', '31', '33', '34', '38', '41', '43', '44', '47', '48', '49', '50', '51', '57', '58', '59', '60', 
                                                     '61', '62', '64', '65', '67', '68', '74', '75', '82', '83', '84', '85', '86', '91', '92', '93', '94', '97']
 
 )
PSO_df1_LGB

In [ ]:
X_train_LGB_PSO, X_test_LGB_PSO, y_train_LGB_PSO,y_test_LGB_PSO= train_test_split(PSO_df1_LGB,train_labels_encoded, test_size=0.2, random_state=0)

In [ ]:
PSO_LGB= lgb.LGBMClassifier(bagging_fraction=0.5, bagging_frequency=5, feature_fraction=0.5,
               max_depth=10, min_data_in_leaf=110, num_leaves=1200,
               random_state=0)    
PSO_LGB.fit(X_train_LGB_PSO, y_train_LGB_PSO.ravel())
PSO_predictions_LGB = PSO_LGB.predict(X_test_LGB_PSO)
print(classification_report(y_test_LGB_PSO,PSO_predictions_LGB))

# GO + RandomForest

In [ ]:
from zoofs import GeneticOptimization
# create object of algorithm
GO_algo_object=GeneticOptimization(objective_function_topass,n_iteration=20,
                                       population_size=40,minimize=False)

In [ ]:
RFC_object_GO = GO_algo_object.fit(RandomForestClassifier(bootstrap = grid_search_RF.best_params_['bootstrap'],
                                          max_features = grid_search_RF.best_params_['max_features'],
                                          n_estimators = grid_search_RF.best_params_['n_estimators'],
                                          criterion = grid_search_RF.best_params_['criterion'],random_state = 1),X_train1, y_train1, X_test1, y_test1,verbose=True)

#plot your results


In [ ]:
print(list(GO_algo_object.best_feature_list))

In [ ]:
GO_algo_object.plot_history()

In [ ]:
GO_df1_RFC= pd.DataFrame(data_rescaled_df, columns=['2', '3', '7', '9', '11', '12', '14', '15', '17', '19', '32', '33', '34', '38', '41',
                                                    '42', '43', '47', '57', '59', '61', '64', '65', '66', '67', '68', '69', '71', '73', '74', 
                                                    '75', '77', '78', '89', '92', '94', '96', '97']
 
 )
GO_df1_RFC

In [ ]:
X_train_RFC_GO, X_test_RFC_GO, y_train_RFC_GO,y_test_RFC_GO= train_test_split(GO_df1_RFC,train_labels_encoded, test_size=0.2, random_state=0)

In [ ]:
GO_RFC= RandomForestClassifier(bootstrap=False, max_features='log2', n_estimators=40,
                       random_state=1)   
GO_RFC.fit(X_train_RFC_GO, y_train_RFC_GO.ravel())
GO_predictions_RFC = GO_RFC.predict(X_test_RFC_GO)
print(classification_report(y_test_RFC_GO,GO_predictions_RFC))

# GO +KNN

In [ ]:
KNN_object_GO = GO_algo_object.fit(KNeighborsClassifier(n_neighbors = grid_search_KNN.best_params_['n_neighbors'],
                                          weights = grid_search_KNN.best_params_['weights'],
                                          metric = grid_search_KNN.best_params_['metric']),X_train1, y_train1, X_test1, y_test1,verbose=True)
#plot your results


In [ ]:
print(list(GO_algo_object.best_feature_list))

In [ ]:
GO_algo_object.plot_history()

In [ ]:
GO_df1_KNN= pd.DataFrame(data_rescaled_df, columns=['0', '1', '3', '6', '7', '8', '11', '13', '18', '24', '25', '26', '31', '32', '33', '36', '39', 
                                                    '43', '45', '47', '49', '50', '51', '52', '54', '56', '62', '66', '67', '79', '81', '83', '84', 
                                                    '88', '89', '92', '93', '95']
 
 )
GO_df1_KNN

In [ ]:
X_train_KNN_GO, X_test_KNN_GO, y_train_KNN_GO,y_test_KNN_GO= train_test_split(GO_df1_KNN,train_labels_encoded, test_size=0.2, random_state=0)

In [ ]:
GO_KNN = KNeighborsClassifier(n_neighbors=6, weights='distance')   
GO_KNN.fit(X_train_KNN_GO, y_train_KNN_GO.ravel())
GO_predictions_KNN = GO_KNN.predict(X_test_KNN_GO)
print(classification_report(y_test_KNN_GO,GO_predictions_KNN))

# GO + SVM

In [ ]:
SVC_object_GO = GO_algo_object.fit(SVC(C = grid_search_SVC.best_params_['C'],
                                          gamma = grid_search_SVC.best_params_['gamma'],
                                          kernel = grid_search_SVC.best_params_['kernel']),X_train1, y_train1, X_test1, y_test1,verbose=True)

#plot your results


In [ ]:
print(list(GO_algo_object.best_feature_list))

In [ ]:
GO_algo_object.plot_history()

In [ ]:
GO_df1_SVC= pd.DataFrame(data_rescaled_df, columns=['1', '3', '5', '7', '8', '9', '11', '12', '14', '15', '18', '20', '21', '25', '29', '30',
                                                    '32', '33', '34', '35', '36', '37', '38', '39', '41', '46', '48', '50', '51', '54', '55', 
                                                    '56', '58', '62', '63', '67', '68', '69', '71', '74', '75', '76', '78', '79', '81', '83',
                                                    '84', '85', '87', '91', '94', '95', '96']
 
 )
GO_df1_SVC

In [ ]:
from sklearn.model_selection import train_test_split
X_train_SVC_GO, X_test_SVC_GO, y_train_SVC_GO,y_test_SVC_GO= train_test_split(GO_df1_SVC,train_labels_encoded, test_size=0.2, random_state=0)

In [ ]:
GO_SVC= SVC(C=100, gamma=0.001, random_state=0) 
GO_SVC.fit(X_train_SVC_GO, y_train_SVC_GO.ravel())
GO_predictions_SVC = GO_SVC.predict(X_test_SVC_GO)
print(classification_report(y_test_SVC_GO,GO_predictions_SVC))

# GO + LGB

In [ ]:
LGB_object_GO = GO_algo_object.fit(lgb.LGBMClassifier(bagging_fraction = grid_search_LGB.best_params_['bagging_fraction'],
                                          bagging_frequency = grid_search_LGB.best_params_['bagging_frequency'],
                                          feature_fraction = grid_search_LGB.best_params_['feature_fraction'],
                                          max_depth = grid_search_LGB.best_params_['max_depth'],
                                          min_data_in_leaf=grid_search_LGB.best_params_['min_data_in_leaf'],
                                          num_leaves=grid_search_LGB.best_params_['num_leaves'],
                                          random_state = 0),X_train1, y_train1, X_test1, y_test1,verbose=True)

#plot your results


In [ ]:
print(list(GO_algo_object.best_feature_list))

In [ ]:
GO_algo_object.plot_history()

In [ ]:
GO_df1_LGB= pd.DataFrame(data_rescaled_df, columns=['0', '2', '3', '4', '6', '7', '10', '12', '13', '15', '17', '19', '21', '22', '23', '24', '25', 
                                                    '27', '28', '30', '33', '36', '37', '49', '50', '51', '52', '53', '54', '56', '57', '59', '60',
                                                    '63', '65', '67', '71', '74', '77', '79', '80', '81', '82', '85', '86', '88', '89', '91', '92', '95']
 
 )
GO_df1_LGB

In [ ]:
from sklearn.model_selection import train_test_split
X_train_LGB_GO, X_test_LGB_GO, y_train_LGB_GO,y_test_LGB_GO= train_test_split(GO_df1_LGB,train_labels_encoded, test_size=0.2, random_state=0)

In [ ]:
GO_LGB= lgb.LGBMClassifier(bagging_fraction=0.5, bagging_frequency=5, feature_fraction=0.5,
               max_depth=10, min_data_in_leaf=110, num_leaves=1200,
               random_state=0)  
GO_LGB.fit(X_train_LGB_GO, y_train_LGB_GO.ravel())
GO_predictions_LGB= GO_LGB.predict(X_test_LGB_GO)
print(classification_report(y_test_LGB_GO,GO_predictions_LGB))